# Advent of Code 2024 Solutions

## 1

In [37]:
from tqdm import tqdm

In [38]:
# part 1
import numpy as np
with open("inputs/input1.txt") as file:
    lines = [line.strip("\n").split() for line in file.readlines()]

lines = np.array(lines).astype("int64")
left = lines[:, 0]
right = lines[:, 1]

ordered = np.vstack([
    np.sort(left),
    np.sort(right)
]).T

difs = np.abs(ordered[:, 0] - ordered[:, 1])
sum(difs)

3574690

In [39]:
# part 2
mults = []
for num in ordered[:, 0]:
    right_occs = sum(ordered[:, 1] == num)
    mults.append(right_occs)
mults = np.array(mults)
sum(mults * ordered[:, 0])

22565391

## 2

In [40]:
# part 1
import numpy as np

def is_safe(report):
    n = report[0]
    difs = []
    for num in report[1:]:
        difs.append(n - num)
        n = num
    difs = np.array(difs)
    if sum(difs > 0) != len(difs) and sum(difs < 0) != len(difs):
        return False
    if sum(np.abs(difs) >= 1) == len(difs) and sum(np.abs(difs) <= 3) == len(difs):
        return True
    return False

with open("inputs/input2.txt") as file:
    reports = [line.strip("\n").split() for line in file.readlines()]

safety_vec = []
for report in reports:
    for i in range(len(report)):
        report[i] = int(report[i])
    safety_vec.append(is_safe(report))
sum(safety_vec)

631

In [41]:
# part 2
from itertools import combinations
def pd_is_safe(report):
    combs = combinations(report, len(report) - 1)
    subsafeties = [is_safe(comb) for comb in combs]
    return True in subsafeties

safety_vec2 = []
for report in reports:
    safety_vec2.append(pd_is_safe(report))
sum(safety_vec2)

665

## 3

In [42]:
# part 1
import re
import numpy as np

def find_multiply(garbled_str):
    mults = re.findall(r"mul\([0-9]+,[0-9]+\)", garbled_str)
    nums = np.array([re.findall(r"[0-9]+", mult) for mult in mults]).astype("int64")
    product = nums[:, 0] * nums[:, 1]
    return sum(product)

with open("inputs/input3.txt") as file:
    lines = [line.strip("\n") for line in file.readlines()]
total = "".join(lines)
find_multiply(total)

187825547

In [43]:
# part 2
def find_multiply_conditional(garbled_str):
    instructions = re.findall(r"mul\([0-9]+,[0-9]+\)|do\(\)|don't\(\)", garbled_str)
    act = True
    valid_instructions = []
    for instruction in instructions:
        if instruction == "don't()":
            act = False
            continue
        elif instruction == "do()":
            act = True
            continue
        elif act:
            valid_instructions.append(instruction)
    nums = np.array([re.findall(r"[0-9]+", mult) for mult in valid_instructions]).astype("int64")
    product = nums[:, 0] * nums[:, 1]
    return sum(product)

find_multiply_conditional(total)

85508223

## 4

In [44]:
# part 1
import numpy as np
with open("inputs/input4.txt") as file:
    ltr_grid = np.array([[letter for letter in line.strip("\n")] for line in file.readlines()])

def get_vecs(start, grid_dims):
    change_vec = np.array([1, 2, 3])
    row_vec = (np.ones(3) * start[0]).astype("int64")
    col_vec = (np.ones(3) * start[1]).astype("int64")
    up = list(zip(row_vec - change_vec, col_vec))
    right = list(zip(row_vec, col_vec + change_vec))
    down = list(zip(row_vec + change_vec, col_vec))
    left = list(zip(row_vec, col_vec - change_vec))
    up_right = list(zip(row_vec - change_vec, col_vec + change_vec))
    down_right = list(zip(row_vec + change_vec, col_vec + change_vec))
    down_left = list(zip(row_vec + change_vec, col_vec - change_vec))
    up_left = list(zip(row_vec - change_vec, col_vec - change_vec))

    directions = [up, up_right, right, down_right, down, down_left, left, up_left]
    valid_directions = []
    for dir_set in directions:
        for i, (row, col) in enumerate(dir_set):
            if row < 0 or col < 0 or row >= grid_dims[0] or col >= grid_dims[1]:
                i = 0
                break
        if i == 2:
            valid_directions.append(dir_set)
    return valid_directions


def find_starts(grid, character):
    starts = []
    for i, row in enumerate(grid):
        for j, letter in enumerate(row):
            if letter == character:
                starts.append((i, j))
    return starts

starts = find_starts(ltr_grid, "X")
count = 0
for start in starts:
    vecs = get_vecs(start, ltr_grid.shape)
    # cursed triple list comprehension
    letters = ["".join(letterset) for letterset in [[ltr_grid[i, j] for i, j in vec] for vec in vecs]]
    count += letters.count("MAS")
count

2534

In [45]:
# part 2
def is_x_mas(position, grid):
    if position[0] - 1 < 0 or \
            position[0] + 1 >= grid.shape[0] or \
            position[1] - 1 < 0 or \
            position[1] + 1 >= grid.shape[1]:
        return False
    upleft = grid[position[0] - 1, position[1] - 1]
    upright = grid[position[0] - 1, position[1] + 1]
    downright = grid[position[0] + 1, position[1] + 1]
    downleft = grid[position[0] + 1, position[1] - 1]

    set1 = {upleft, downright}
    set2 = {upright, downleft}

    if set1 == {"S", "M"} and set1 == set2:
        return True
    return False

tfs = []
starts = find_starts(ltr_grid, "A")
for start in starts:
    tfs.append(is_x_mas(start, ltr_grid))
sum(tfs)

1866

## 5

In [46]:
# part 1
import numpy as np
with open("inputs/input5.txt") as file:
    lines = [line.strip("\n") for line in file.readlines()]
rules = np.array([line.strip("\n").split("|") for line in lines if "|" in line]).astype("int64")
updates = [[int(x) for x in line.strip("\n").split(",")] for line in lines if "|" not in line and len(line) != 0]

def get_afters(pgnum):
    return rules[rules[:, 0] == pgnum, 1]

def is_valid(update):
    for i, num in enumerate(update):
        afters = get_afters(num)
        for after in afters:
            if after in update[:i]:
                return False
    return True

valid_updates = [update for update in updates if is_valid(update)]
centers = [update[len(update) // 2] for update in valid_updates]
print(sum(centers))

5762


In [47]:
# part 2: recursion!
invalid_updates = [update for update in updates if not is_valid(update)]

def correct_update(update):
    if is_valid(update):
        return update
    for i, num in enumerate(update):
        corrected = update
        afters = get_afters(num)
        for after in afters:
            if after in update[:i]:
                loc = update.index(after)
                #weird python thing allows me to slice beyond the indices of an array
                corrected = corrected[:loc] + [num] + corrected[loc:i] + corrected[i + 1:]
                return correct_update(corrected)

corrected = [correct_update(update) for update in invalid_updates]
centers = [update[len(update) // 2] for update in corrected]
print(sum(centers))

4130


# 6

In [48]:
# part 1
import numpy as np
with open("inputs/input6.txt") as file:
    lines = np.array([[letter for letter in line.strip("\n")] for line in file.readlines()])

def chart_path(grid):
    pathed_grid = grid.copy()
    # let's recycle an older function here
    loc = list(find_starts(pathed_grid, "^")[0])
    sym = "^"
    while 0 < loc[0] < pathed_grid.shape[0] and 0 < loc[1] < pathed_grid.shape[1]:
        try:
            if sym == "^":
                if pathed_grid[loc[0] - 1, loc[1]] == "#":
                    sym = ">"
                else:
                    pathed_grid[loc[0], loc[1]] = "X"
                    loc[0] -= 1
            elif sym == ">":
                if pathed_grid[loc[0], loc[1] + 1] == "#":
                    sym = "v"
                else:
                    pathed_grid[loc[0], loc[1]] = "X"
                    loc[1] += 1
            elif sym == "v":
                if pathed_grid[loc[0] + 1, loc[1]] == "#":
                    sym = "<"
                else:
                    pathed_grid[loc[0], loc[1]] = "X"
                    loc[0] += 1
            elif sym == "<":
                if pathed_grid[loc[0], loc[1] - 1] == "#":
                    sym = "^"
                else:
                    pathed_grid[loc[0], loc[1]] = "X"
                    loc[1] -= 1
        except IndexError:
            break
    pathed_grid[loc[0], loc[1]] = "X" # mark last position
    return pathed_grid

# recycle it again
pathed = chart_path(lines)
xs = find_starts(pathed, "X")
print(len(xs))

4819


In [49]:
# part 2
def is_looped(grid):
    loc = list(find_starts(grid, "^")[0])
    sym = "^"
    path_dict = {}
    while 0 < loc[0] < grid.shape[0] and 0 < loc[1] < grid.shape[1]:
        if tuple(loc) in path_dict and sym in path_dict[tuple(loc)]:
            return 1
        elif tuple(loc) in path_dict and sym not in path_dict[tuple(loc)]:
            path_dict[tuple(loc)].append(sym)
        elif tuple(loc) not in path_dict:
            path_dict[tuple(loc)] = [sym]
        try:
            if sym == "^":
                if grid[loc[0] - 1, loc[1]] == "#":
                    sym = ">"
                else:
                    loc[0] -= 1
            elif sym == ">":
                if grid[loc[0], loc[1] + 1] == "#":
                    sym = "v"
                else:
                    loc[1] += 1
            elif sym == "v":
                if grid[loc[0] + 1, loc[1]] == "#":
                    sym = "<"
                else:
                    loc[0] += 1
            elif sym == "<":
                if grid[loc[0], loc[1] - 1] == "#":
                    sym = "^"
                else:
                    loc[1] -= 1
        except IndexError:
            break
    return 0


looped = []
possible_obstruction_locations = xs
for i, j in possible_obstruction_locations:
    mapcopy = lines.copy()
    if mapcopy[i, j] == "^":
        continue
    mapcopy[i, j] = "#"
    looped.append(is_looped(mapcopy))
print(sum(looped))

1796
